# Исследование надежности заемщиков


In [2]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

In [3]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Категории:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создание новых категорий:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данных

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data_pivot_children = data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])
data_pivot_children['mean'] = (data_pivot_children['mean']*100).round(decimals=2)
data_pivot_children['share'] = (data_pivot_children['sum']/data_pivot_children['count']).round(decimals=2)
data_pivot_children


,count,sum,mean,share
,debt,debt,debt,
children,,,,
0,14091,1063,7.54,0.08
1,4808,444,9.23,0.09
2,2052,194,9.45,0.09
3,330,27,8.18,0.08
4,41,4,9.76,0.10
5,9,0,0.00,0.00


<div class="alert alert-info"> <b>Комментарий:</b> 

Count считает нам количество человек в зависимости от количества детей. Sum показывает количество должников. То есть из 14091 человек, у которых нет детей, 1063 человека являются должниками. 
Можно заметить, что больше всего людей, которые имеют задолженность, не имеют детей. Чем больше детей, тем ниже количество должников. При этом людей, у которых в семье детей больше, тоже меньше. Так, у людей с 5 детьми в семье, отсутсвуют задолжности. 
Таким же образом у людей с тремя детьми меньше задолжностей, чем у семей с 1 и 2 детьми. Однако столбец share говорит, что как бездетные, так и семьи до 4 детей включительно примерно на одном уровне задолжности, о чем также говорит и среднее (столбец mean).

Вывод: не совсем хватает данных по многодетным семьям, поэтому нельзя сказать, что связь есть. Также не следует опираться только на sum и count :)
    




</div>

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data_pivot_family_status = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count', 'sum', 'mean'])
data_pivot_family_status['mean'] = (data_pivot_family_status['mean']*100).round(decimals=2)
data_pivot_family_status['share'] = (data_pivot_family_status['sum']/data_pivot_family_status['count']).round(decimals=2)
data_pivot_family_status

,count,sum,mean,share
,debt,debt,debt,
family_status,,,,
Не женат / не замужем,2796,273,9.76,0.10
в разводе,1189,84,7.06,0.07
вдовец / вдова,951,63,6.62,0.07
гражданский брак,4134,385,9.31,0.09
женат / замужем,12261,927,7.56,0.08


**Вывод:** 

<div class="alert alert-info"> <b>Комментарии:</b> 

Count также считает нам количество людей в зависимости от семейного положения. Sum количество должников.
    
    Соответственно доля должников выше у людей, в статусе не женат/не замужем (10%). Близка к данному значению доля должников, состоящих в гражаднском браке (9%). Меньше всех не возвращают кредит в срок люди в разводе (7%), те кто в статусе женат/замужем (8%) и вдовец/вдова (7%)

    




</div>

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data_pivot_total_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
data_pivot_total_income['mean'] = (data_pivot_total_income['mean']*100).round(decimals=2)
data_pivot_total_income['share'] = (data_pivot_total_income['sum']/data_pivot_total_income['count']).round(decimals=2)
data_pivot_total_income


,count,sum,mean,share
,debt,debt,debt,
total_income_category,,,,
A,25,2,8.00,0.08
B,5014,354,7.06,0.07
C,15921,1353,8.50,0.08
D,349,21,6.02,0.06
E,22,2,9.09,0.09


<div class="alert alert-info"> <b>Комментарии:</b> 
    
Соответственно больше всего людей зарабатывают от 50000 до 200000. Поэтому среди этой группы больше всего должников. Посмотрев на mean и share, то примерно все находятся в одинаковой зависимости. В среднем каждый 7-8 человек является должником. Меньше всего должников в группе D с более низким доходом от 30000 до 50000 тысяч. На следующей ступени люди из группы B с высоким доходом от 200000 до 1000000. В группе E из 22 человек двое являются должниками.
Можно сделать вывод, что зависимость слабая. Возможно необходимо больше данных о должниках. 



</div>

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data_pivot_purpose_category = data.pivot_table(
    index=['purpose_category'],
    values='debt',
    aggfunc=['count', 'sum', 'mean']
)

data_pivot_purpose_category['mean'] = (data_pivot_purpose_category['mean']*100).round(decimals=2)

data_pivot_purpose_category['share'] = (
    data_pivot_purpose_category['sum']/
    data_pivot_purpose_category['count']
).round(decimals=2)

data_pivot_purpose_category



,count,sum,mean,share
,debt,debt,debt,
purpose_category,,,,
операции с автомобилем,4279,400,9.35,0.09
операции с недвижимостью,10751,780,7.26,0.07
получение образования,3988,369,9.25,0.09
проведение свадьбы,2313,183,7.91,0.08


<div class="alert alert-info"> <b>Комментарии:</b> 

    
Соответственно больше всего людей берут кредит, целью которого это операции с недвижимостью. Если посмотреть на отношение должников ко всем людям из определенной категории, то зависимость слабая, т.к доли должников мало различаются между собой. Немного чаще оказываются в долгах те, кто берут кредит на операции с автомобилем, получение образования и проведение свадьбы. Меньше всего на операции с недвижимостью.



</div>

### Шаг 4: общий вывод.

<div class="alert alert-info"> <b>Комментарии:</b> 

    
Финальный вывод:
В ходе данного проекта было проведено исследование, с целью выявления влияния разных факторов на факт погашения кредита в срок. Входные данные - статистика о платежеспособности клиентов. 

В ходе проекта необходимо было ответить на следующие вопросы:
    
    Есть ли зависимость между количеством детей и возвратом кредита в срок?
    Есть ли зависимость между семейным положением и возвратом кредита в срок?
    Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    Как разные цели кредита влияют на его возврат в срок?

1. Ответ: слабая зависимость
    Доля семей без детей и должников - 0.08. Доля семей с 3 и 4 детьми в семье - 0.08 и 0.1 соотвественно. 
       
    
2. Ответ: зависимость есть
    Доля должников выше у людей, в статусе не женат/не замужем (10%). Близка к данному значению доля должников, состоящих в гражаднском браке (9%). Меньше всех не возвращают кредит в срок люди в разводе (7%), те кто в статусе женат/замужем (8%) и вдовец/вдова (7%)
       
    
3. Ответ: слабая зависимость

    0–30000 — 'E';
    30001–50000 — 'D';
    50001–200000 — 'C';
    200001–1000000 — 'B';
    1000001 и выше — 'A'.
    
    Доля должников выше в группах по уровню дохода: A - 8%, C - 8%, E - 9%. Чуть ниже в группах (B - 7%, D - 6%). Нельзя сказать, что из этого можно сделать какую-то зависимость. 
    
    
    
4. Ответ: слабая зависимость
    Доли должников по категория не сильно различаются между собой.
    
    




</div>